In [ ]:
import pandas as pd
import numpy as np
# import mplhep as hep
from tqdm import tqdm
import  matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.max_columns', 150)

import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split


: 

In [9]:
# filecode = '_InfAwar_test'
oc = torch.tensor(np.load(f'/vols/cms/hw423/Data/Week14/octest_InfA_RD_DPrmvd_Bld.npy'))
def hess_to_tensor(H):
    hess_elements = []
    for i in range(len(H)):
        for j in range(len(H)):
            hess_elements.append(H[i][j].reshape(1))
    return torch.cat(hess_elements).reshape(len(H),len(H))
pred =  torch.argmax(oc,dim=1)

In [15]:
label = pd.read_pickle('/vols/cms/hw423/Data/Week14/Label_DPrmvd.pickle')

In [23]:
oc.size()

torch.Size([435646, 7])

In [5]:
#Define the Net
class Net(nn.Module):
    def __init__(self, n_features=40, nodes=[10,10], output_nodes=10):
        super(Net, self).__init__()
        # Build network
        n_nodes = [n_features] + nodes + [output_nodes]
        self.layers = nn.ModuleList()
        for i in range(len(n_nodes)-1):
            self.layers.append(nn.Linear(n_nodes[i], n_nodes[i+1]))
            self.layers.append(nn.ReLU())

    def forward(self, x):
        out = self.layers[0](x)
        for layer in self.layers[1:]:
            out = layer(out)
        # Apply softmax
        return torch.softmax(out, dim=1)

In [6]:
class InfAwareLoss(nn.Module):
    def __init__(self,weight,):
        super(InfAwareLoss, self).__init__()
        self.weight = weight
    
    def forward(self,input,target):
        # Input = torch.tensor(input)
        # Target = torch.tensor(target,dtype=torch.int8)
        label = torch.argmax(target,dim=1)
        pred = torch.argmax(input,dim=1)
        # plt.hist(pred)
        plt.show()
        weight = torch.tensor(self.weight.values)
        d = input.size()[1]
        cm = torch.zeros(7,d)
        for t, p, w in zip(label.view(-1), pred.view(-1), weight.view(-1)):
            cm[p,t] += w
        cm =cm[1:, :]
        O = cm.sum(dim=1)
        # print(cm)
        # print(O)
        def NLL(mu):
            mu0 =torch.tensor([1.0])
            theta = torch.cat((mu0,mu))
            return -(O@(torch.log(cm@theta))-(cm@theta).sum())
        mu = torch.tensor([1.0,1.0,1.0,1.0,1.0,1.0],requires_grad=True)
        hess = torch.func.hessian(NLL)(mu)
        I = torch.inverse(hess_to_tensor(hess))
        loss = torch.trace(I)**0.5 
        return loss.clone().detach().requires_grad_(True)

In [7]:
pred

NameError: name 'pred' is not defined